In [4]:
from probability import probability
import warnings
import numpy as np
from numba import jit
from scipy.integrate import quad

# Suppress only IntegrationWarning
warnings.filterwarnings("ignore")

In [5]:
@jit(nopython=True)
def integrand(x, xw, xp, ri, Mi):
    p = np.exp(-xp * ri * (xw - x))
    integrand_value = np.exp(x) * p * ((1 - p) ** (Mi - 1))
    return integrand_value


def probability(xw, xp, ri, Mi):
    factor_up_front = 1.0 / (np.exp(xw) - 1)
    integral = quad(
        integrand,
        0,
        xw,
        args=(xw, xp, ri, Mi),
        limit=200,
        epsabs=1e-10,
        epsrel=1e-10,
    )[0]
    return factor_up_front * integral

For a given set of $(x_w, x_p, r_i, M_i)$, import 'probability' function from 'probability.py' file. This function calculates probability of proliferation for a single TCR indexed by $i$ given the waiting time $x_w$, proliferation rate $x_p$, propensity to proliferate $r_i$ and observed/expected clone size $M_i$.

In [11]:
probability = np.vectorize(probability)
def new_neg_likelihood(
    params, fixed_xw, clone_count_values, scaled_kr_values, verbose=False
):
    # params is now just x2, fixed_x1 is provided separately
    xp = params[0]  # params is now a 1D array with just x2
    xw = fixed_xw
    n = len(clone_count_values)
    xp_values = np.full(n, xp)
    xw_values = np.full(n, xw)
    
    probs = probability(xw_values, xp_values, scaled_kr_values, clone_count_values)
    # Replace zero values with the smallest positive value allowed in Python
    smallest_positive_value = np.finfo(float).eps
    probabilities = np.where(probs == 0, smallest_positive_value, probs)
    sum_log_probs = np.sum(np.log(probabilities))
    neg_sum = -sum_log_probs

    if verbose:
        print(f"Neg-logL: {neg_sum:.8f}")
        print(f"xp: {xp:.8f}")
        print(f"=" * 80)
    return neg_sum

In [12]:
new_neg_likelihood([0.2], 100, range(1, 11), [0.1]*10, verbose=True)

Neg-logL: 131.25685931
xp: 0.20000000


np.float64(131.25685931382287)

In [ ]:
The negative log-likelihood is defined in minimization.py.
